In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders,get_dataloaders_subword
from utils_subword import load_sentencepiece_model,sp_detokenize_with_specials
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
BATCH_SIZE = 64# butuh lebih banyak update 
DATA_PATH = "dataset/"  
DATA_PATH_SPM = "dataset/spm_files"  
SAVE_DIR = "saved"
SRC_VOCAB_SIZE = 5000
TGT_VOCAB_SIZE = 5000
N_LAYERS = 2
N_HEADS = 1
D_MODEL = 128
FFN_HIDDEN = D_MODEL*4
DROPOUT = 0.1
EPOCHS = 300

sp = load_sentencepiece_model(DATA_PATH_SPM, model_prefix="spm_model")
train_loader, val_loader, test_loader = get_dataloaders_subword(
    sp=sp,
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)

encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)


SentencePiece model loaded from dataset/spm_files/spm_model.model
TrainData - Max 'min' sentence length: 76
TrainData - Max 'eng' sentence length: 107
TestData - Max 'min' sentence length: 61
TestData - Max 'eng' sentence length: 75
ValidData - Max 'min' sentence length: 71
ValidData - Max 'eng' sentence length: 80
Number of examples in train_dataset: 800
Number of examples in valid_dataset: 100
Number of examples in test_dataset: 100
Model initialized on: cuda


In [3]:
tokens = {
    "Padding": utils.PAD_TOKEN,
    "Start of Sequence": utils.SOS_TOKEN,
    "End of Sequence": utils.EOS_TOKEN,
    "Unknown": utils.UNK_TOKEN
}
for i, batch in enumerate(train_loader):
    if i < 8:
        src = batch["src"]
        tgt = batch["tgt"]
        ss,fss=src[0,:],src.shape
        st,fst=tgt[0,:],tgt.shape
        print(fss,fst)
    for name, token in tokens.items():
        print(f"{name}: {token}")
    else:
        break

torch.Size([64, 107]) torch.Size([64, 107])
Padding: 0
Start of Sequence: 1
End of Sequence: 2
Unknown: 3


testing input , is it correct

In [4]:
data_iter = iter(train_loader)
for i in range(1):
    batch = next(data_iter)
    print(batch["tgt"])


tensor([[   1, 3837,   71,  ...,    0,    0,    0],
        [   1,  244, 4998,  ...,    0,    0,    0],
        [   1, 3676, 4989,  ...,   15, 4981,  392],
        ...,
        [   1,  910,  453,  ...,    0,    0,    0],
        [   1,  910,   63,  ...,   13, 1106, 4998],
        [   1, 2814,   13,  ...,    0,    0,    0]])


In [5]:
for i, batch in enumerate(train_loader):
    if i < 1:
        src_batch = batch['src'].to(device)  # shape: [batch_size, seq_len]
        tgt_batch = batch['tgt'].to(device)  # shape: [batch_size, seq_len]
        batch_size = src_batch.size(0)
        num_examples = min(5, batch_size)
        for j in range(num_examples):
            src_tokens = src_batch[j].tolist()
            tgt_tokens = tgt_batch[j].tolist()
            src_sentence = sp_detokenize_with_specials(sp, src_tokens)
            tgt_sentence = sp_detokenize_with_specials(sp, tgt_tokens)
            print(f"Example {j+1}:")
            print("Source:", src_sentence)
            print("Target:", tgt_sentence)
            print("-" * 50)
    break


Example 1:
Source: awak belabelaan dari bogor jam demi makan sup buntutnyo tapi awak tunggu dari jam alun bukak juo akhirnyo awak tibo jam pasan sup buntut alun ado jo alasan tadi malam rami jadi chefnyo masih tepar wow alasan macam apo mode itu beda bana wakatu wak makan tahun nan lalu yo tampek sangaik kumuah bantuak indak niek punyo usaho
Target: i went throgh a hour trip from bogor just to eat that otail soup but i had been waiting from and they werent open yet in the end i decided to come at ordered the otail soup but it was not available because last night was so hectic so the chef w
--------------------------------------------------
Example 2:
Source: haragonyo sangai tajangkau dan menu makanannyo cukuik kakinian tampek makannyo didekor dengan elok dan unik
Target: the price is incredibly affordable and the menu is hip and cool the restos decor is arranged well and uniuely
--------------------------------------------------
Example 3:
Source: awak ko sanak istri wak sadang maabia

In [6]:
for i, batch in enumerate(train_loader):
    if i < 1:
        src_batch = batch['src'].to(device)  # shape: [batch_size, seq_len]
        tgt_batch = batch['tgt'].to(device)  # shape: [batch_size, seq_len]
        batch_size = src_batch.size(0)
        num_examples = min(5, batch_size)
        for j in range(num_examples):
            src_tokens = src_batch[j].tolist()
            tgt_tokens = tgt_batch[j].tolist()
            # Use the SentencePiece detokenization function.
            src_sentence = sp_detokenize_with_specials(sp, src_tokens)
            tgt_sentence = sp_detokenize_with_specials(sp, tgt_tokens)
            print(f"Example {j+1}:")
            print("Source:", src_sentence)
            print("Target:", tgt_sentence)
            print("-" * 50)
    break


Example 1:
Source: maaf indak nomer awak se nan baitu awak tanya ka sadolah nan pakai di daerah situ mangatoan hal nan samo masih indak nio ngaku jo kalau memang sinyalnyo nan buruak
Target: ecuse me but its not just my number eperiencing that ive asked everyone who uses it in that area and they all said the same thing still dont wanna admit that your signal is just that bad
--------------------------------------------------
Example 2:
Source: ndak salah lai ko kafe modern salah satu favorit warga depok untuak makan bakumpua seriusan tampeknyo nyaman soalnyo
Target: there is no mistake that this modern cafe is one of the favourites for depok citiens to go to for dining or gathering seriously its cu the place is that comfy
--------------------------------------------------
Example 3:
Source: untuak mananyoan kredit mandiri finance bisa di siko lo
Target: you can also ask uestions about mandiri finance credit here
--------------------------------------------------
Example 4:
Source: stei

In [7]:
import os
import torch
from tqdm import tqdm
os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf") 
best_model_path = None
atl = {}
avl = {}
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=True, total=len(train_loader))
    for batch in train_bar:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    atl[epoch]=avg_train_loss
    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    avl[epoch]=avg_val_loss
    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best_sp.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")

utils.plot_loss(atl, avl, SAVE_DIR,"loss_sp")


Epoch 1/300


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 68.27it/s, loss=8.6690]


[Epoch 1] Train Loss: 8.6665 | Val Loss: 8.6440
  -> New best model saved at saved/best_sp.pt
Epoch 2/300


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 72.34it/s, loss=8.6230]


[Epoch 2] Train Loss: 8.6226 | Val Loss: 8.5985
  -> New best model saved at saved/best_sp.pt
Epoch 3/300


...

[Epoch 298] Train Loss: 4.6929 | Val Loss: 4.7246
  -> New best model saved at saved/best_sp.pt
Epoch 299/300


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 63.12it/s, loss=4.6417]


[Epoch 299] Train Loss: 4.6898 | Val Loss: 4.7213
  -> New best model saved at saved/best_sp.pt
Epoch 300/300


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 63.09it/s, loss=4.6381]


[Epoch 300] Train Loss: 4.6816 | Val Loss: 4.7181
  -> New best model saved at saved/best_sp.pt
Loss plot saved to saved/loss_sp.png
